<center><h1> Sample notebook to extract SOCA-LIGHT-2023 profile from BGC-Argo Temp-Sal profiles and Satellite Ocean Color Matchups </h1></center>



******************************************

<div style="text-align: right"><i> Renosh - November 2023 </i></div>
<div style="text-align: right"><i> Email: renosh.pr@imev-mer.fr </i></div>
<div style="text-align: right"><i> Email: pr.renosh@gmail.com </i></div>

In [5]:
import warnings
warnings.filterwarnings('ignore')

## 1) Load all libraries

In [6]:
#######################################################################################
#######################################################################################
# Load all the libraries
import sys
import pathlib
from pathlib import Path
import pickle
import os
import numpy as np
import pandas as pd
from netCDF4 import Dataset as NetCDFFile 
import import_ipynb
from datetime import date, datetime, timedelta
import datetime as dt
# import oceans.sw_extras as swe
import gsw
import matplotlib.pyplot as plt
import timezonefinder, pytz
tf = timezonefinder.TimezoneFinder()
#######################################################################################
#######################################################################################

*************

## 2) Import SOCA-Light model functions

In [7]:
import import_ipynb
from FUNCTIONS.OTHER_FUNCTIONS import rad_date, rad_LC_hour, derive_ZPD,find_MLD
from FUNCTIONS.PAR_ADJUSTED_FUNCTION import PAR_ADJUSTED_profiles
from FUNCTIONS.ED380_ADJUSTED_FUNCTION import ED380_ADJUSTED_profiles
from FUNCTIONS.ED412_ADJUSTED_FUNCTION import ED412_ADJUSTED_profiles
from FUNCTIONS.ED490_ADJUSTED_FUNCTION import ED490_ADJUSTED_profiles

**********************

## 3) Extraction of SOCA-light profiles

### Extract light profiles from input file and BGC-Argo file
********************************************************************
<div style="text-align: left"><i> The path of the BGC-Argo file need to be precised in input file </i></div>
<div style="text-align: left"><i> The output files will be saved in the corresponding output folder for each variable. </i></div>
<div style="text-align: left"><i> For example SOCA-PAR outputs stored in OUTPUTS/PAR folder </i></div>
<div style="text-align: left"><i> If the path are correctly mention in '/INPUTS/inputs.xlsx' user will get outputs in corresponding folders. No need to change anything in this NOTEBOOK </i></div>

In [8]:
## The inputs from excel file a sample file (INPUTS/inputs.xlsx) 
## The output file name as DAC_WMO_CYCLE_ED490.txt format this can change as per users choice

#######################################################################################
# define output path (WHERE YOU WANT TO SAVE THE OUTPUT FILES)
# PAR_OUT_PATH = pathlib.Path('/Users/renoshpr/Documents/BLUE_CLOUD/SOCA_CHL_2020/NOTEBOOK/PAR_ADJUSTED_MODEL_2022/PAR_ADJUSTED_Extraction_2023_51_depths/Output')
curr_work_dir = Path.cwd()
PAR_OUT_PATH = (''.join([str(curr_work_dir),'/OUTPUTS/PAR']))
ED380_OUT_PATH = (''.join([str(curr_work_dir),'/OUTPUTS/ED380']))
ED412_OUT_PATH = (''.join([str(curr_work_dir),'/OUTPUTS/ED412']))
ED490_OUT_PATH = (''.join([str(curr_work_dir),'/OUTPUTS/ED490']))

#######################################################################################
# READ INPUTS FROM EXCEL FILE
#
input_file_name = (''.join([str(curr_work_dir),'/INPUTS/inputs.xlsx']))
# input_file_name = 'inputs.xlsx'
Inputs=pd.read_excel(str(input_file_name))

for i in np.arange(len(Inputs)):
    BGC_ARGO_NC_FILE = Inputs.BGC_ARGO_NC_FILE[i]
    RHO412 = Inputs.RHO412[i]
    RHO443 = Inputs.RHO443[i]
    RHO490 = Inputs.RHO490[i]
    RHO555 = Inputs.RHO555[i]
    RHO670 = Inputs.RHO670[i]
    PAR = Inputs.PAR[i]
    KD490 = Inputs.KD490[i]
    PAR_output_file = (''.join([str(PAR_OUT_PATH),'/',str(Inputs.DAC[i]),'_',str(Inputs.WMO[i]),'_',str(Inputs.N_CYCLE[i]),'_PAR.txt']))
    ED380_output_file = (''.join([str(ED380_OUT_PATH),'/',str(Inputs.DAC[i]),'_',str(Inputs.WMO[i]),'_',str(Inputs.N_CYCLE[i]),'_ED380.txt']))
    ED412_output_file = (''.join([str(ED412_OUT_PATH),'/',str(Inputs.DAC[i]),'_',str(Inputs.WMO[i]),'_',str(Inputs.N_CYCLE[i]),'_ED412.txt']))
    ED490_output_file = (''.join([str(ED490_OUT_PATH),'/',str(Inputs.DAC[i]),'_',str(Inputs.WMO[i]),'_',str(Inputs.N_CYCLE[i]),'_ED490.txt']))
    
    #######################################################################################
    
    if os.path.isfile(BGC_ARGO_NC_FILE): # check Argo file exist
        # LOAD THE BGC_ARGO file
        #######################################################################################
        # ADD the path of the Argo NC file
        NC= NetCDFFile(str(BGC_ARGO_NC_FILE))
    
        # Check Parameter and its Data mode
        PARAMETER=NC.variables['PARAMETER']
        PARAMETER = ''.join(str(s, encoding='UTF-8') for s in PARAMETER)
        PARAMETER = PARAMETER.split()

        DATA_M = NC.variables['PARAMETER_DATA_MODE']
        DATA_M  = ''.join(str(s, encoding='UTF-8') for s in DATA_M)
        
        # PRES_MODE = DATA_M[PARAMETER.index('PRES')]
        TEMP_MODE = DATA_M[PARAMETER.index('TEMP')]
        PSAL_MODE = DATA_M[PARAMETER.index('PSAL')]
        
        if ((TEMP_MODE=='D') and (PSAL_MODE=='D')): # check temp sal datamode
            #######################################################################################
            ##### EXTRACTING TEMP SAL PRES same depths FROM NC ARGO FILE ##########################
            #######################################################################################
            pres_1 = NC.variables['PRES_ADJUSTED'][:]
            temp_1 = NC.variables['TEMP_ADJUSTED'][:]
            sal_1 = NC.variables['PSAL_ADJUSTED'][:]             
            pres_qc_1 = NC.variables['PRES_ADJUSTED_QC'][:]
            temp_qc_1 = NC.variables['TEMP_ADJUSTED_QC'][:]
            sal_qc_1 = NC.variables['PSAL_ADJUSTED_QC'][:]
            LAT = float(NC.variables['LATITUDE'][:])
            LON = float(NC.variables['LONGITUDE'][:])

            pres1 = pres_1.filled(fill_value=np.nan)
            pres_qc = pres_qc_1.filled(fill_value=np.nan)
            temp1 = temp_1.filled(fill_value=np.nan)
            temp_qc = temp_qc_1.filled(fill_value=np.nan)
            sal1 = sal_1.filled(fill_value=np.nan)
            sal_qc = sal_qc_1.filled(fill_value=np.nan)

            pres1[pres_qc == b'3']= np.nan
            pres1[pres_qc == b'4']= np.nan
            temp1[temp_qc == b'3']= np.nan
            temp1[temp_qc == b'4']= np.nan
            sal1[sal_qc == b'3']= np.nan
            sal1[sal_qc == b'4']= np.nan

            pres2 = pres1[(~np.isnan(pres1)) & (~np.isnan(sal1)) & (~np.isnan(temp1))]
            temp2 = temp1[(~np.isnan(pres1)) & (~np.isnan(sal1)) & (~np.isnan(temp1))]
            sal2 = sal1[(~np.isnan(pres1)) & (~np.isnan(sal1)) & (~np.isnan(temp1))]

            ########################################################################################
            # Declare output depths
            #Output daepth values 0, 5 10, 15, ... 250
            depth_output = np.linspace(0,250,51)
            #######################################################################################
    
            pres3=pres2[pres2<=250] # Light profile computed if T/S have at least 10 good points in 0 to 250 m water column
    
            pres3=pres2[pres2<=250] # Light profile computed if T/S have at least 15 good points in 0 to 250 m water column
            pres4=pres2[pres2<=50] # Light profile computed if T/S have at least 5 good points in 0 to 50 m water column
            if (len(pres3)>=15) & (len(pres4)>=5): # selected T/S profiles qualified above criterias
                #######################################################################################
                ##### EXTRACTING MLD ##################################################################
                #######################################################################################               
                # Derive Density Sigma0 and MLD from Temperature and Salinity profile
                # No need to change anything in this section by the user
                SA = gsw.SA_from_SP(sal2, pres2, LON, LAT) # Abslute salinity from practical salinity
                CT = gsw.CT_from_t(SA, temp2, pres2) # Conservative Temperature from in-situ sea water temperature
                dens = gsw.density.sigma0(SA,CT) # Potential density
                MLD_1 = find_MLD(Z=pres2, dens=dens, threshold = 0.03)
                # print('MLD is :', MLD_1,'m')
        
                #######################################################################################
                ##### EXTRACTING DOY FROM NC ARGO FILE ################################################
                #######################################################################################
                JULD = NC.variables['JULD'][:]
                # #Get day, month, year from juld
                ref_date = '19500101000000'
                JULD_n = int(JULD*86400)
                date_prof = pd.to_datetime(ref_date,format='%Y%m%d%H%M%S') + timedelta(seconds=JULD_n)
                year_prof = date_prof.strftime('%Y')
                month_prof = date_prof.strftime('%m')
                day_prof = date_prof.strftime('%d')
                DOY = int(date_prof.strftime('%j'))
                # print('DOY is:' ,DOY)
                # No need to change anything in this section by the user
                sin_doy = np.sin(rad_date(DOY))
                cos_doy = np.cos(rad_date(DOY))

                ########################################################################################################
                # Extract local time from the nc file and convert into radiance and then transform into Sine and Cosine
                ########################################################################################################
                timezone_str = tf.certain_timezone_at(lat=LAT, lng=LON)
                timezone = pytz.timezone(timezone_str)
                # date_prof is in GMT time scale
                LC = date_prof + timezone.utcoffset(date_prof) # +
                GMT_Hour = date_prof.strftime('%H')
                LC_Hour = LC.strftime('%H') 
    
                GMT_Hour_Min = date_prof.strftime('%H:%M')
                LC_Hour_Min = LC.strftime('%H:%M')  
                LC_Hour_Deci = (int(LC_Hour_Min[0:2]) + int(LC_Hour_Min[3:])/60)

                # LC_hour = 12
                sin_LC_hour = np.sin(rad_LC_hour(LC_Hour_Deci))
                cos_LC_hour = np.cos(rad_LC_hour(LC_Hour_Deci))
                ####################################################################################### 
                #######################################################################################
                # No need to change anything in this section by the user
                # HOW to use this function
        
                if(not np.isnan(MLD_1) and not np.isnan(PAR) and not np.isnan(KD490)and
                   not np.isnan(RHO412) and not np.isnan(RHO443) and not np.isnan(RHO490) and
                   not np.isnan(RHO555) and not np.isnan(RHO670)):
                    PAR_ADJUSTED=PAR_ADJUSTED_profiles(PAR=PAR, MLD=MLD_1, ZPD=derive_ZPD(float(KD490)), RHO_WN_412=RHO412, RHO_WN_443=RHO443,
                                                       RHO_WN_490=RHO490, RHO_WN_555=RHO555, RHO_WN_670=RHO670, sin_doy=sin_doy,
                                                       cos_doy=cos_doy, sin_LC_hour=sin_LC_hour, cos_LC_hour=cos_LC_hour,
                                                       temp=temp2, sal=sal2, depths_argo=pres2, depth_output=depth_output)[0]
                    ED380_ADJUSTED=ED380_ADJUSTED_profiles(PAR=PAR, MLD=MLD_1, ZPD=derive_ZPD(float(KD490)), RHO_WN_412=RHO412, RHO_WN_443=RHO443,
                                                       RHO_WN_490=RHO490, RHO_WN_555=RHO555, RHO_WN_670=RHO670, sin_doy=sin_doy,
                                                       cos_doy=cos_doy, sin_LC_hour=sin_LC_hour, cos_LC_hour=cos_LC_hour,
                                                       temp=temp2, sal=sal2, depths_argo=pres2, depth_output=depth_output)[0]
                    ED412_ADJUSTED=ED412_ADJUSTED_profiles(PAR=PAR, MLD=MLD_1, ZPD=derive_ZPD(float(KD490)), RHO_WN_412=RHO412, RHO_WN_443=RHO443,
                                                       RHO_WN_490=RHO490, RHO_WN_555=RHO555, RHO_WN_670=RHO670, sin_doy=sin_doy,
                                                       cos_doy=cos_doy, sin_LC_hour=sin_LC_hour, cos_LC_hour=cos_LC_hour,
                                                       temp=temp2, sal=sal2, depths_argo=pres2, depth_output=depth_output)[0]
                    ED490_ADJUSTED=ED490_ADJUSTED_profiles(PAR=PAR, MLD=MLD_1, ZPD=derive_ZPD(float(KD490)), RHO_WN_412=RHO412, RHO_WN_443=RHO443,
                                                       RHO_WN_490=RHO490, RHO_WN_555=RHO555, RHO_WN_670=RHO670, sin_doy=sin_doy,
                                                       cos_doy=cos_doy, sin_LC_hour=sin_LC_hour, cos_LC_hour=cos_LC_hour,
                                                       temp=temp2, sal=sal2, depths_argo=pres2, depth_output=depth_output)[0]                   
        
                else:
                    PAR_ADJUSTED = np.nan
                    ED380_ADJUSTED = np.nan
                    ED412_ADJUSTED = np.nan
                    ED490_ADJUSTED = np.nan
            else: # If temp/Sal criteria fails
                PAR_ADJUSTED = np.nan
                ED380_ADJUSTED = np.nan
                ED412_ADJUSTED = np.nan
                ED490_ADJUSTED = np.nan
        else: # Temp and Sal not in delayed mode
            PAR_ADJUSTED = np.nan
            ED380_ADJUSTED = np.nan
            ED412_ADJUSTED = np.nan
            ED490_ADJUSTED = np.nan
    else: # If there is no BGC Argo file
        PAR_ADJUSTED = np.nan
        ED380_ADJUSTED = np.nan
        ED412_ADJUSTED = np.nan
        ED490_ADJUSTED = np.nan
              
    # #########################################################################################
    # ##########       
    db_par = {'depth': depth_output, 'PAR_ADJUSTED': PAR_ADJUSTED} 
    db_par = pd.DataFrame(db_par)
    db_par= db_par.round({'depth': 0, 'PAR_ADJUSTED': 4}) # Output adjusted to 4 decimal places
    
    db_ed380 = {'depth': depth_output, 'ED380_ADJUSTED': ED380_ADJUSTED} 
    db_ed380 = pd.DataFrame(db_ed380)
    db_ed380 = db_ed380.round({'depth': 0, 'ED380_ADJUSTED': 5}) # Output adjusted to 5 decimal places
    
    db_ed412 = {'depth': depth_output, 'ED412_ADJUSTED': ED412_ADJUSTED} 
    db_ed412 = pd.DataFrame(db_ed412)
    db_ed412 = db_ed412.round({'depth': 0, 'ED412_ADJUSTED': 5}) # Output adjusted to 5 decimal places
    
    db_ed490 = {'depth': depth_output, 'ED490_ADJUSTED': ED490_ADJUSTED} 
    db_ed490 = pd.DataFrame(db_ed490)
    db_ed490 = db_ed490.round({'depth': 0, 'ED490_ADJUSTED': 5}) # Output adjusted to 5 decimal places    
    
    ###########################################################################################
    db_par.to_csv(PAR_output_file, sep=',', index= False)
    db_ed380.to_csv(ED380_output_file, sep=',', index= False)   
    db_ed412.to_csv(ED412_output_file, sep=',', index= False)   
    db_ed490.to_csv(ED490_output_file, sep=',', index= False)   
       
    print(i)
##############################################################################################

0


*******